# MAT281 - Tarea 2

**Indicaciones**:

* En los **Ejercicio 1-8** puedes utilizar tanto `matplotlib` como `altair` según te parezca más conveniente o cómodo, en ambos casos cada gráfico debe tener elementos mínimos como:
    - Título
    - Nombre de los ejes, leyendas, etc. en formato _amigable_/_humano_, por ejemplo, si la columna del dataframe en cuestión tiene por nombre `casos_confirmados` se espera que el eje del gráfico tenga por nombre `Casos confirmados`.
    - Colores adecuados al tipo de datos.
    - Un tamaño adecuado para ver con facilidad en una pantalla con resolución HD o FullHD.
    - Cada vez que no se cumplan alguna de estos requerimientos se descontará __1 punto__ de la nota final.

* Para el **Ejercicio 9** es obligación utilizar `altair`.
* Cada ejercicio debe estar acompañado con una celda con comentarios o análisis que puedas desprender de los gráficos.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
#alt.data_transformers.enable('data_server')
alt.data_transformers.disable_max_rows()
alt.themes.enable('opaque')

%matplotlib inline

**COVID-19 en Chile** 

En esta tarea exploraremos los datos de Covid-19 en Chile a profundidad. Las siguientes celdas cargarán los datos a utilizar en tu sesión. Es importante que leas la documentación de cada conjunto de datos para comprender las columnas.

In [2]:
start_date = pd.to_datetime("2020-04-13")

In [36]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)

confirmados.head()

,poblacion,casos_confirmados,fecha,region_id,region,provincia_id,provincia,comuna_id,comuna,tasa
0,247552.0,270.0,2020-04-27,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,109.067994
1,247552.0,9138.0,2020-10-12,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,3691.345657
2,247552.0,596.0,2020-05-29,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,240.757497
3,247552.0,6131.0,2020-08-17,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,2476.651370
4,247552.0,353.0,2020-05-11,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,142.596303


In [4]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto19
activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos.head()

,region,codigo_region,comuna,codigo_comuna,poblacion,fecha,casos_activos
0,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-04-13,88.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-04-13,0.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-04-13,0.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-04-13,0.0
4,Tarapaca,1,Alto Hospicio,1107.0,129999.0,2020-04-13,8.0


In [5]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto14
fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos.head()

,region,fecha,fallecidos
374,Arica y Parinacota,2020-04-13,1.0
375,Tarapacá,2020-04-13,0.0
376,Antofagasta,2020-04-13,1.0
377,Atacama,2020-04-13,0.0
378,Coquimbo,2020-04-13,0.0


In [6]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10
fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo.head()

,grupo_de_edad,fecha,fallecidos
28,0-39,2020-04-13,3
29,40-49,2020-04-13,2
30,50-59,2020-04-13,3
31,60-69,2020-04-13,12
32,70-79,2020-04-13,29


## Ejercicio 1

(10 puntos)

Mostrar cantidad de fallecidos la fecha por cada grupo etáreo.

In [7]:
alt.Chart(fallecidos_etareo).mark_bar().encode(
    x=alt.X('grupo_de_edad:N',axis=alt.Axis(title= 'grupo de edad')),
    y='fallecidos:Q',
).properties(
title = 'fallecidos a la fecha por grupo etario',
width=500,
height=300
)

alt.Chart(...)

**Comentarios:** Se puede observar que los grupos etarios de mayor edad tienen mayor cantida de fallecidos, siendo el máximo el rango entre 70-79 años con más de cuatro mil fallecidos a la fecha. Mientras en la población de menor edad su cantidad de fallecidos apenas supera los 500 fallecidos a la fecha.


## Ejercicio 2

(10 puntos)

¿Qué tan variable es la población de las comunas de Chile? Considera utilizar un gráfico que resuma de buena forma la información sin agregar la variable de región o provincia.

In [41]:
Poblacion_variable=alt.Chart(activos).mark_bar().encode(
    x=alt.X('comuna:N',title="Comuna"),
    y=alt.Y('poblacion',title="Poblacion"),
).properties(
    title="Variacion de poblacion entre comunas"
)  

Poblacion_variable

alt.Chart(...)

**Comentarios:** En el gráfico se puede la mayoría de la poblacion entre comunas es menor a cien mil. Sin embargo, se observan comunas con una gran diferencia de poblacion, con más de 600.000 habitantes en Puente Alto. Esto muestra la gran variabilidad de población entre comunas

## Ejercicio 3

(10 puntos)

Mostrar evolución y comparación de los fallecimientos entre distintos grupos etáreos, pero que al mismo tiempo sea fácil identificar la cantidad de fallecidos total en cada fecha.

In [10]:
alt.Chart(fallecidos_etareo).mark_line().encode(
    x='fecha:T',
    y='fallecidos:Q',
    color=alt.Color('grupo_de_edad:N',title='grupo etario')
).properties(
    title='Evolución de fallecimientos por grupos etario',
    width=500,
    height=300
)

alt.Chart(...)

**Comentarios:** Podemos ver que los grupos con mayor edad (entre 60-89) presentar una curva de crecimiento mayor de fallecidos respecto al resto, de la misma forma en los grupos de edad menores(0-49) la curva de incremento de fallecidos a lo largo del tiempo no es tan pronunciada.

## Ejercicio 4

(10 puntos)

Mostrar en tres gráficos la evolución de casos confirmados, evolución de fallecimientos y evolución de casos activos.

In [11]:
casos_confirmados= alt.Chart(confirmados).mark_bar().encode(
    x='fecha:T',
    y=alt.Y('casos_confirmados:Q',title='casos confirmados'),
).properties(
    width=500,
    height=300,
    title='Evolución de casos confirmados'
)
fallecimientos= alt.Chart(fallecidos).mark_bar().encode(
    x='fecha:T',
    y='fallecidos:Q',
).properties(
    width=500,
    height=300,
    title='Evolución de fallecimientos'
)

casos_activos= alt.Chart(activos).mark_bar().encode(
    x='fecha:T',
    y=alt.Y('casos_activos:Q',title='casos activos'),
).properties(
    width=500,
    height=300,
    title="Evolucion de casos activos"
)
casos_confirmados | fallecimientos | casos_activos

alt.HConcatChart(...)

**Comentarios:** Podemos ver que las curvas de crecimiento entre casos confirmados y fallecimientos es similar, donde se puede notar que a partir de julio la curva incrementó drásticamente. Por otro lado, la curva de evolucion de casos activos fue variando, incrementó de forma rápida hasta julio y luego fue disminuyendo hasta establizarse en noviembre, con menos de 500 casos activos.

## Ejercicio 5

(10 puntos)

Comparar la tasa de incidencia entre las regiones a lo largo del tiempo.

In [39]:
tasa_region=(confirmados.groupby(["region","fecha"])
       .agg(poblacion=("poblacion","sum"),casos_confirmados=("casos_confirmados","sum"))
       .reset_index()
)
tasa_region["tasa"]= tasa_region["casos_confirmados"]*100000/tasa_region["poblacion"]
tasa_region.head()

,region,fecha,poblacion,casos_confirmados,tasa
0,Antofagasta,2020-04-13,691854.0,149.0,21.536336
1,Antofagasta,2020-04-15,691854.0,176.0,25.438893
2,Antofagasta,2020-04-17,691854.0,211.0,30.497764
3,Antofagasta,2020-04-20,691854.0,264.0,38.158340
4,Antofagasta,2020-04-24,691854.0,368.0,53.190413


In [40]:
alt.Chart(tasa_region).mark_line().encode(
    x='fecha:T',
    y=alt.Y('tasa:Q',title="Tasa"),
    color='region'
).properties(
    width=500,
    height=300
)

alt.Chart(...)

**Comentarios:** Se puede ver que en la mayoria de regiones la curva no creció tan drásticamente, llegando a un máximo sobre los dos mil. Por otro lado, se pueden ver 4 regiones con una curva pronunciada, llegando a valores sobre los cuatro mil. Finalmente hay una región con una curva de crecimiento más pronunciada (Magallanes) llegando a los nueve mil

## Ejercicio 6

(10 puntos)

¿Hay alguna conclusión que puedas obtener rápidamente al graficar un _scatter plot_ con los casos confirmados y tasa de incidencia de cada comuna para los días 13 de abril y 6 de noviembre del 2020? Además, colorea cada punto según la región a la que pertenece y considera si es útil en el gráfico que el tamaño sea proporcional a la población.

In [13]:
start_date = pd.to_datetime("2020-04-13")
end_date= pd.to_datetime("2020-11-6")
trece_abril = alt.Chart(confirmados.loc[lambda x:x["fecha"]==start_date]).mark_point().encode(
    x=alt.X('casos_confirmados:Q',title='casos confirmados'),
    y='tasa:Q',
    color="region"
).properties(
    width=500,
    height=300,
    title="13 de abril del 2020"
)
seis_noviembre= alt.Chart(confirmados.loc[lambda x:x["fecha"]==end_date]).mark_point().encode(
    x=alt.X('casos_confirmados:Q',title='casos confirmados'),
    y='tasa:Q',
    color='region'
).properties(
    title='6 de Noviembre del 2020',
    width=500,
    height=300
)
trece_abril | seis_noviembre

alt.HConcatChart(...)

**Comentarios:**  En ambos gráficos se puede ver que al aumentar la cantidad de casos confirmados aumenta su tasa de incidencia. sin embargo, la tasa es mayor para comunas con menos habitantes, pues, estas se pueden ver aún más afectadas con incrementos pequeños de casos confirmados. Por lo mismo sería útil que el tamaño sea proporcional a la población, para así distinguir estos casos.

## Ejercicio 7

(10 puntos)

1. Grafica la evolución de los casos activos de cada comuna en un solo gráfico. 
2. Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.

Entrega los pros y contras de cada uno de estos enfoques.

In [14]:
alt.Chart(activos).mark_line().encode(
    x='fecha:T',
    y=alt.Y('casos_activos:Q',title="casos activos"),
    color='comuna'
).properties(
    width=500,
    height=300,
    title="Evolucion de casos activos por comuna"
)

alt.Chart(...)

In [15]:
alt.Chart(activos).mark_line().encode(
    x='fecha:T',
    y=alt.Y('casos_activos:Q',title="casos activos"),
    column='region',
    color='comuna'
).properties(
    width=500,
    height=300,
    title="Evolución de casos activos por comuna"
)

alt.Chart(...)

**Comentarios:** En el primer gráfico es dificil distinguir cada comuna, por lo que solo se pude obtener información y comparar las comunas con mayor cantidad de casos activos y ver donde se ubica la mayoria de comunas. Por otro lado, en el segundo es más fácil distinguir las comunas por región, pero dado la gran cantidad de gráficos no se puede tener un visualizacion global como en el primero. Además en ambos gráficos, dado la gran cantidad de comunas es dificil distinguir entre los colores.

## Ejercicio 8

(10 puntos)

Hacer un gráfico que permita comparar rápidamente entre regiones su promedio de casos activos , máximo de casos confirmados y fallecidos. Utiliza los valores reales y apoyarlos con colores.

Se adjunta el diccionario `region_names` con tal de reemplazar los nombres de las regiones en los datos `fallecidos` para poder unir con los otros datos. 

In [16]:
region_names = {
    "Araucanía": "La Araucanía",
    "Aysén": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
}

In [17]:
fallecidos["region"]=fallecidos["region"].replace(region_names)

In [18]:
region= (confirmados.rename(columns={"region_id":"codigo_region"}).groupby(["region","codigo_region"])
           .agg(max_casos_confirmados=("casos_confirmados","max")).reset_index()
           .merge(activos.groupby("codigo_region").agg(promedio_casos_activos=("casos_activos","mean")),how="inner",on="codigo_region")
           .merge(fallecidos.groupby("region").agg(max_fallecidos= ("fallecidos","max")).reset_index(),how="inner",on="region")
           .melt(id_vars =["region","codigo_region"])
)
region.head()

,region,codigo_region,variable,value
0,Antofagasta,2.0,max_casos_confirmados,12831.0
1,Arica y Parinacota,15.0,max_casos_confirmados,10530.0
2,Atacama,3.0,max_casos_confirmados,4820.0
3,Aysén del General Carlos Ibáñez del Campo,11.0,max_casos_confirmados,1086.0
4,Biobío,8.0,max_casos_confirmados,6401.0


In [19]:
alt.Chart(region).mark_line().encode(
    x=alt.X('region:N',title="Región"),
    y=alt.Y('value:Q',title="Valor"),
    color=alt.Color('variable:N',title="Variable")
).properties(
    width=600,
    height=400,
    title="Promedio casos activos, maximo de fallecidos y casos confirmados entre regiones"
)

alt.Chart(...)

**Comentarios:** Se puede ver que la región Metropolitana tiene el maximo de fallecios, casos confirmados y su promedio de casos activos es levemente mayor. Para el resto de regiones tanto la cantidad máxima de fallecidos como su promedio de casos activos son bajos, teniendo algunos leves aumentos las regiones de Biobio, Libertador General Bernardo O'Higgins y Valparaíso. Finalmente la cantidad de maximo de casos confirmado fue la más variable entre regiones.

## Ejercicio 9


En este ejercicio buscaremos realizar un mini-dashboard respecto al estado de los casos de COVID-19 en Chile, por lo tanto utilizaremos haremos uso de datos geográficos de manera operacional (es decir, no nos preocuparemos de proyecciones en mapas ni nada por el estilo), lo único es que debes instalar `geopandas` en tu ambiente virtual y no olvidar actualizarlo en tu `environment.yml` para luego subirlo a tu repositorio de GitHub.

Con tu ambiente activo (`conda activate mat281`) basta con ejecutar `conda install -c conda-forge geopandas` para instalar `geopandas`.

In [20]:
import geopandas as gpd
from pathlib import Path

In [21]:
shp_filepath = Path().resolve().parent / "data" / "regiones_chile.shp"
regiones = gpd.read_file(shp_filepath)
regiones.head()

,Region,objectid,cir_sena,codregion,area_km,st_area_sh,st_length_,geometry
0,Región Metropolitana de Santiago,1092,7,13,15392.030737,2.225204e+10,1.064253e+06,"POLYGON ((-7873736.745 -3885505.642, -7873695...."
1,Región de Antofagasta,1086,3,2,126071.431981,1.508452e+11,2.516112e+06,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
2,Región de Arica y Parinacota,1084,1,15,16866.819844,1.886869e+10,7.505296e+05,"POLYGON ((-7727277.278 -1997230.768, -7726464...."
3,Región de Atacama,1089,4,3,75661.248635,9.643906e+10,2.401741e+06,"MULTIPOLYGON (((-7900342.628 -3153340.296, -79..."
4,Región de Aysén del Gral.Ibañez del Campo,1088,14,11,106703.377369,2.242743e+11,4.144481e+07,"MULTIPOLYGON (((-8208500.834 -5733817.475, -82..."


In [22]:
type(regiones)

geopandas.geodataframe.GeoDataFrame

Lo único que tienes que saber es que un `GeoDataFrame` es idéntico a un `DataFrame` salvo que debe poseer una columna llamada `geometry` caracterice los elementros geométricos, que en este casos son polígonos con los límites de las regiones de Chile. 

Para graficar mapas en Altair se debe usar `mark_geoshape`, además, para no preocuparnos de las proyecciones si o si debes declarar lo siguiente que se muestra en la siguiente celda en las propiedades del gráfico. El resto es igual a cualquier otro gráfico de Altair.

In [23]:
alt.Chart(regiones).mark_geoshape().encode(
).properties(
    projection={'type': 'identity', 'reflectY': True},
    width=250,
    height=600
)

alt.Chart(...)

### Ejercicio 9.1

(10 puntos)

Define el `DataFrame` con el nombre `casos_geo` tal que tenga las columnas

* `region`
* `codigo_region`
* `fecha`
* `poblacion`
* `casos_confirmados`
* `tasa`
* `casos_activos`
* `fallecidos`
* `geometry`

Ten mucho cuidado como unes los dataframes `confirmados`, `activos`, `fallecidos` y `regiones`. Idealmente utilizar el código de región, pero en caso que no se encuentren disponibles utilizar el nombre de la región (no olivdar utilizar el diccionario `region_names`).

In [24]:
confirmados_tasa= (confirmados.rename(columns={"region_id":"codigo_region"}).drop(columns={"provincia","provincia_id","comuna","comuna_id"})
       .groupby(["region","codigo_region","fecha"])
       .agg(poblacion=("poblacion","sum"),casos_confirmados=("casos_confirmados","sum"))
       .reset_index()
       .sort_values(["codigo_region","fecha"])
)
confirmados_tasa["tasa"]=confirmados_tasa["casos_confirmados"]*100000/confirmados_tasa["poblacion"]
confirmados_tasa.head()

,region,codigo_region,fecha,poblacion,casos_confirmados,tasa
858,Tarapacá,1.0,2020-04-13,382773.0,52.0,13.585075
859,Tarapacá,1.0,2020-04-15,382773.0,62.0,16.197590
860,Tarapacá,1.0,2020-04-17,382773.0,73.0,19.071356
861,Tarapacá,1.0,2020-04-20,382773.0,93.0,24.296385
862,Tarapacá,1.0,2020-04-24,382773.0,130.0,33.962688


In [25]:
casos_geo = (confirmados_tasa
             .merge(activos.groupby(["codigo_region","fecha"])["casos_activos"].sum().reset_index(),how="inner",on=["codigo_region","fecha"])
             .merge(fallecidos,how="inner",on=["region","fecha"])
             .merge(regiones.rename(columns={"codregion":"codigo_region"})[["codigo_region","geometry"]],how="inner",on="codigo_region")
             .sort_values(["codigo_region","fecha"])
)

casos_geo.head()

,region,codigo_region,fecha,poblacion,casos_confirmados,tasa,casos_activos,fallecidos,geometry
0,Tarapacá,1.0,2020-04-13,382773.0,52.0,13.585075,30.0,0.0,"MULTIPOLYGON (((-7807680.075 -2295867.719, -78..."
1,Tarapacá,1.0,2020-04-15,382773.0,62.0,16.197590,35.0,0.0,"MULTIPOLYGON (((-7807680.075 -2295867.719, -78..."
2,Tarapacá,1.0,2020-04-17,382773.0,73.0,19.071356,38.0,0.0,"MULTIPOLYGON (((-7807680.075 -2295867.719, -78..."
3,Tarapacá,1.0,2020-04-20,382773.0,93.0,24.296385,42.0,0.0,"MULTIPOLYGON (((-7807680.075 -2295867.719, -78..."
4,Tarapacá,1.0,2020-04-24,382773.0,130.0,33.962688,61.0,0.0,"MULTIPOLYGON (((-7807680.075 -2295867.719, -78..."


Ejecuta lo siguiente para convertir el DataFrame anterior en un GeoDataFrames

In [26]:
casos_geo = casos_geo.pipe(lambda x: gpd.GeoDataFrame(x, geometry="geometry"))

### Ejercicio 9.2

(5 puntos)

Modifica la función `covid_chile_chart` tal que reciba una fecha y una columna. Luego, debe filtrar `casos_geo` con registros de la fecha seleccionada y graficar un mapa donde las regiones se colereen según la columna escogida. 

In [27]:
def covid_chile_chart(fecha, col):
    
    fecha = pd.to_datetime(fecha)
    data = casos_geo.loc[lambda x: x["fecha"]==fecha]
    
    chart = alt.Chart(data).mark_geoshape().encode(
        color=col
    ).properties(
        projection={'type': 'identity', 'reflectY': True},
        width=150,
        height=400
    )
    
    chart.display()
    return

Prueba con lo siguiente

In [28]:
fecha = "2020-04-13"
col = "tasa"
covid_chile_chart(fecha, col)

alt.Chart(...)

### Ejercicio 9.3

(5 puntos)

Ahora utilizando `widgets` generaremos el dashboard interactivo. Define lo siguiente:

* col_widget: Un `widgets.Dropdown` donde las opciones a seleccionar sean las columnas `poblacion`, `casos_confirmados`, `tasa`, `casos_activos` y `fallecidos`. Además, el argumento `description` debe ser `Columna`.
* fecha_widget: Un `widgets.DatePicker` donde el argumento `description` sea `Fecha`.
* Ambos widgets deben tener el argumento `continuous_update=False`

In [29]:
import ipywidgets as widgets
from ipywidgets import interactive, interact

In [30]:
col_widget = widgets.Dropdown(
    options=["poblacion","casos_confirmados","tasa","casos_activos","fallecidos"],
    description="Columna",
    continuous_update=False
)

In [31]:
fecha_widget = widgets.DatePicker(
    description="Fecha",
    continuous_update=False
)

Finalmente, haciendo uso de `interactive`, la función `covid_chile_chart` y todos los widgets es posible crear un _dashboard_ interactivo con los datos de Covid-19.

Respira profundo y explora tu creación!

In [32]:
covid_dashboard = interactive(
    covid_chile_chart,
    fecha=fecha_widget,
    col=col_widget
)
covid_dashboard

interactive(children=(DatePicker(value=None, description='Fecha'), Dropdown(description='Columna', options=('p…

**Comentarios:**